# Dodgers Data Pipeline

### Import Required Libraries & Dependencies

In [18]:
import requests
import sqlite3
import time
import argparse
import os
import pandas as pd

In [1]:
!python3 --version

Python 3.11.5


In [20]:
print('requests', requests.__version__)
print(sqlite3.version)
print(argparse.__version__)
print(pd.__version__)
print(os.__version__)

requests 2.31.0
2.6.0
1.1
2.1.4


AttributeError: module 'os' has no attribute '__version__'

In [182]:
!which python

/Users/landonsmith/anaconda3/bin/python


In [184]:
pip install --upgrade ipykernel

  Obtaining dependency information for ipykernel from https://files.pythonhosted.org/packages/b0/71/348c659f0fbbf526ca77fae9fd31d43a3c90d4d60ae3df6d477055ee90d5/ipykernel-6.28.0-py3-none-any.whl.metadata
  Obtaining dependency information for debugpy>=1.6.5 from https://files.pythonhosted.org/packages/64/59/b9c5dbe60c3a3e4e4bb12b1968aa75e98b99ff8b65273a5e7de75cceef75/debugpy-1.8.0-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.1/114.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 9.3 MB/s eta 0:00:0000:01:00:01
  Attempting uninstall: debugpy
    Found existing installation: debugpy 1.5.1
    Uninstalling debugpy-1.5.1:
      Successfully uninstalled debugpy-1.5.1
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 6.19.2
    Uninstalling ipykernel-6.19.2:
      Successfully uninstalled ipykernel-6.19.2

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip insta

### Define Create Table Statements

In [5]:
def delete_all_tables(db_path):
    # Retrieve all table names in the database
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    # Drop each table one by one
    for table_name, in tables:  # comma to unpack tuple
        try:
            cursor.execute(f"DROP TABLE {table_name}")
            print(f"Table {table_name} deleted.")
        except sqlite3.Error as e:
            print(f"An error occurred while deleting table {table_name}: {e}")

In [79]:
delete_all_tables('baseball.db')

In [78]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all results
tables = cursor.fetchall()
tables

[]

In [ ]:
# If you want to print the schema of a specific table
# Replace 'your_table_name' with the actual table name
cursor.execute("PRAGMA table_info('fielding_season_stats')")
schema = cursor.fetchall()
schema

In [22]:
url = 'https://baseballsavant.mlb.com/gf?game_pk=635925'

#Make a GET request to the API
response = requests.get(url)

#Check if the request was successful
if response.status_code == 200:
    #Store JSON data into variable
    data = response.json()
else:
    #Store failed retrievals inside a dictionary with the key being game_pk and value being status code
    failed_to_retrieve[pk] = reponse.status_code

In [23]:
data

{'game_status_code': 'F',
 'game_status': 'F',
 'gamedayType': 'P',
 'gameDate': '10/20/2020',
 'scoreboard': {'gamePk': 635925,
  'linescore': {'currentInning': 9,
   'currentInningOrdinal': '9th',
   'inningState': 'Top',
   'inningHalf': 'Top',
   'isTopInning': True,
   'scheduledInnings': 9,
   'innings': [{'num': 1,
     'ordinalNum': '1st',
     'home': {'runs': 0, 'hits': 0, 'errors': 0, 'leftOnBase': 1},
     'away': {'runs': 0, 'hits': 1, 'errors': 0, 'leftOnBase': 2}},
    {'num': 2,
     'ordinalNum': '2nd',
     'home': {'runs': 0, 'hits': 1, 'errors': 0, 'leftOnBase': 1},
     'away': {'runs': 0, 'hits': 0, 'errors': 0, 'leftOnBase': 0}},
    {'num': 3,
     'ordinalNum': '3rd',
     'home': {'runs': 0, 'hits': 0, 'errors': 0, 'leftOnBase': 1},
     'away': {'runs': 0, 'hits': 0, 'errors': 0, 'leftOnBase': 0}},
    {'num': 4,
     'ordinalNum': '4th',
     'home': {'runs': 2, 'hits': 1, 'errors': 0, 'leftOnBase': 1},
     'away': {'runs': 0, 'hits': 0, 'errors': 0, 'leftO

# SQL Queries

### Which pitchers threw the most pitches in a game? How many pitches did they throw? How many batters did they face?

In [135]:
query = """
with top_pitch_counts as (SELECT game_pk, pitcher_name, MAX(player_total_pitches) as top_pitch_counts 
FROM pitch
GROUP BY 1,2
ORDER BY 3 DESC
LIMIT 5),
ab_count as (SELECT p.game_pk, p.pitcher_name, batter_name, COUNT(*) as ab_count
FROM pitch p
JOIN top_pitch_counts c ON c.game_pk = p.game_pk
WHERE pitch_number = 1
GROUP BY 1,2,3),
batters_faced as (SELECT game_pk, pitcher_name, SUM(ab_count) as batters_faced
FROM ab_count
WHERE pitcher_name IN (SELECT pitcher_name FROM top_pitch_counts)
GROUP BY 1, 2
ORDER BY 3 DESC)
SELECT tc.pitcher_name, top_pitch_counts, batters_faced
FROM top_pitch_counts tc
JOIN batters_faced bf ON tc.pitcher_name = bf.pitcher_name
"""
df = pd.read_sql_query(query, conn)
df

,pitcher_name,top_pitch_counts,batters_faced
0,Tyler Glasnow,112,23
1,Jack Flaherty,110,26
2,Max Fried,109,31
3,Kyle Hendricks,106,28
4,Gerrit Cole,105,27


### Who are the top batters by average exit velocity on balls in play, split by both pitcher handedness and batter handedness? What is their average exit velocity? How many balls in play did they have?

In [173]:
query = '''
SELECT batter_name, p_throws as pitcher_side, stand as batter_side, ROUND(avg(hit_speed), 2) as avg_exit_velo, COUNT(*) as balls_in_play
FROM pitch
WHERE hit_speed NOT NULL AND call_name = 'In Play'
GROUP BY 1,2,3
ORDER BY 4 DESC
LIMIT 5
'''
df = pd.read_sql_query(query, conn)
df

,batter_name,pitcher_side,batter_side,avg_exit_velo,balls_in_play
0,Carlos Santana,L,R,108.70,1
1,Jose Ramirez,L,R,108.50,1
2,Miguel Sano,L,R,107.35,2
3,Brian Anderson,L,R,106.95,2
4,Adam Engel,L,R,106.87,3


### What are the average pitch speed, spin rate, horizontal break, and induced vertical break for each pitch type?

In [178]:
query = '''
SELECT pitch_name, ROUND(AVG(start_speed),2) as avg_speed, ROUND(AVG(spin_rate),2) as avg_spin, ROUND(AVG(breakX),2) as avg_breakX, ROUND(AVG(breakZ),2) as avg_breakZ
FROM pitch
GROUP BY 1
'''
df = pd.read_sql_query(query, conn)
df

,pitch_name,avg_speed,avg_spin,avg_breakX,avg_breakZ
0,4-Seam Fastball,95.00,2364.22,6.72,13.38
1,Changeup,85.86,1775.28,14.05,30.09
2,Curveball,80.07,2663.17,9.36,53.48
3,Cutter,88.97,2417.73,3.67,26.84
4,Fastball,79.79,1706.71,10.43,34.57
5,Knuckle Curve,82.85,2738.32,9.64,53.31
6,Sinker,93.75,2138.81,15.41,23.61
7,Slider,85.67,2493.16,5.67,35.10
8,Slow Curve,76.10,2441.00,8.00,59.00
9,Slurve,82.25,2725.30,15.26,41.20


In [21]:
delete_all_tables('baseballsavant.db')

Table fielding_season_stats deleted.
Table batter_season_stats deleted.
Table pitcher_season_stats deleted.
Table player_descriptions deleted.
Table team_data deleted.
Table games deleted.
Table pitch deleted.
Table linescore deleted.
Table officials deleted.
Table game_wpa deleted.


In [82]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_names = cursor.fetchall()
table_names = [n[0] for n in table_names]
table_names

['fielding_season_stats',
 'batter_season_stats',
 'pitcher_season_stats',
 'player_descriptions',
 'team_data',
 'games',
 'pitch',
 'linescore',
 'officials',
 'game_wpa']

In [31]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_names = cursor.fetchall()
table_names = [n[0] for n in table_names]
if 'games' in table_names:
    game_pk_check = 'SELECT EXISTS(SELECT 1 FROM games WHERE game_pk = ? LIMIT 1)'
    cursor.execute(game_pk_check, (635889,))
    result = cursor.fetchone()[0]
if result == 1:
    print(f'The game_pk {pk} is already present! Ingestion aborted. ')
    #continue
else:
    print(f'Adding game_pk {pk} now!')
        

Adding game_pk 635886 now!


In [ ]:
table_names

In [ ]:
result

In [ ]:
conn.rollback()

In [ ]:
# Commit the changes and close the connection
conn.commit()
conn.close()

In [81]:
conn = sqlite3.connect('baseball.db')

#Create Cursor Object
cursor = conn.cursor()


In [37]:
os.path.getsize('baseball.db') / 1024**2

8.2578125